In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

--2024-11-14 17:19:14--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2024-11-14 17:19:14--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  30.7MB/s    in 6.4s    

2024-11-14 17:19:21 (36.8 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
import torchvision
from torchvision import transforms

train_data_path = 'tiny-imagenet-200/train'
val_data_path = 'tiny-imagenet-200/val'
test_data_path = 'tiny-imagenet-200/test'

In [ ]:
transforms = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225] )
])

In [ ]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)

In [ ]:
import torch

batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Assuming train_set is the Dataset object, such as datasets.CIFAR10 or datasets.ImageFolder
print("Number of samples:", len(train_data))  # Total number of samples

# Access a single sample
sample, label = train_data[0]
print("Sample shape:", sample.shape if isinstance(sample, torch.Tensor) else "Varies by dataset")
print("Label:", label)

# If the dataset has classes (e.g., ImageFolder or CIFAR10), you can access them like this:
if hasattr(train_data, 'classes'):
    print("Classes:", train_data.classes)  # List of class names
    print("Number of classes:", len(train_data.classes))


Number of samples: 100000
Sample shape: torch.Size([3, 64, 64])
Label: 0
Classes: ['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814

In [ ]:
from torch import nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(1288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = x.view(-1, 1288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

simplenet = SimpleNet()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [ ]:
import torch
import torch.nn.functional as F

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        # Training Loop
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)  # Move both inputs and targets to device

            # Forward pass
            output = model(inputs)
            loss = loss_fn(output, targets)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            training_loss += loss.item()

        # Calculate average training loss
        training_loss /= len(train_loader)

        # Validation Loop
        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():  # Disable gradient calculations for validation
            for batch in val_loader:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)

                # Forward pass
                output = model(inputs)
                loss = loss_fn(output, targets)
                valid_loss += loss.item()

                # Calculate accuracy
                predictions = torch.argmax(F.softmax(output, dim=1), dim=1)
                correct = predictions.eq(targets)
                num_correct += correct.sum().item()
                num_examples += correct.shape[0]

        # Calculate average validation loss
        valid_loss /= len(val_loader)

        # Print statistics
        accuracy = num_correct / num_examples
        print(f'Epoch: {epoch + 1}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {accuracy:.2f}')

device = "cpu"
epochs = 20

train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs, device)

RuntimeError: shape '[-1, 1288]' is invalid for input of size 786432

In [ ]:
torch.save(simplenet, "/tmp/simplenet")

In [ ]:
### loading the model
simplenet = torch.load("/tmp/simplenet")

<ipython-input-11-20950181f453>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  simplenet = torch.load("/tmp/simplenet")


In [ ]:
## saving the state dict
torch.save(simplenet.state_dict(), "/content/simplenet")

In [ ]:
simplenet = SimpleNet()
simple_state_dict = torch.load("/content/simplenet")
simplenet.load_state_dict(simple_state_dict)

<ipython-input-16-851b1ebe79aa>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  simple_state_dict = torch.load("/content/simplenet")


<All keys matched successfully>